Source: https://huggingface.co/blog/train-decision-transformers

In [4]:
from dataclasses import dataclass

from datasets import load_dataset
from transformers import DecisionTransformerConfig, DecisionTransformerModel, Trainer, TrainingArguments
import numpy as np

In [2]:
dataset = load_dataset("edbeeching/decision_transformer_gym_replay", "halfcheetah-expert-v2")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset decision_transformer_gym_replay downloaded and prepared to /Users/education/.cache/huggingface/datasets/edbeeching___decision_transformer_gym_replay/halfcheetah-expert-v2/1.1.0/8a8520881dd09333b04f5a116dd47931314ec44b72597bce8675d40ccd42ad88. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
@dataclass
class DecisionTransformerGymDataCollator:
    return_tensors: str = "pt"
    max_len: int = 20 #subsets of the episode we use for training
    state_dim: int = 17  # size of state space
    act_dim: int = 6  # size of action space
    max_ep_len: int = 1000 # max episode length in the dataset
    scale: float = 1000.0  # normalization of rewards/returns
    state_mean: np.array = None  # to store state means
    state_std: np.array = None  # to store state stds
    p_sample: np.array = None  # a distribution to take account trajectory lengths
    n_traj: int = 0 # to store the number of trajectories in the dataset
    
    def __init__(self, dataset):
        self.act_dim = len(dataset[0]["actions"][0])
        self.state_dim = len(dataset[0]["observations"][0])
        self.dataset = dataset
        
        states = []
        traj_lens = []
        
        for obs in dataset["observations"]:
            states.extend(obs)
            traj_lens.append(len(obs))
        
        self.n_traj = len(traj_lens)
        states = np.vstack(states)
        

In [6]:
class TrainableDT(DecisionTransformerModel):
    def __init__(self, config):
        super().__init__(config)
    
    def forward(self, **kwargs):
        output = super().forward(self, **kwargs)
        
        action_preds = output[1]
        action_targets = kwargs["actions"]
        attention_mask= kwargs["attention_mask"]